<a href="https://colab.research.google.com/github/Giuse1093/CSI_Project4/blob/main/data_preprocessing4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# --- CELLA 1: INSTALLAZIONE E SETUP ---
!pip install -q flwr pandas numpy scipy scikit-learn

import os
import zipfile
import ast
import numpy as np
import pandas as pd
from scipy.fft import fft
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings

warnings.filterwarnings('ignore')

# Decompressione Dataset
if not os.path.exists("CSV_train") and os.path.exists("CSV_train.zip"):
    print("Decompressione CSV_train.zip...")
    with zipfile.ZipFile("CSV_train.zip", 'r') as zip_ref:
        zip_ref.extractall("CSV_train")
    print("Fatto!")
elif not os.path.exists("CSV_train.zip"):
    print("⚠️ ATTENZIONE: Carica il file 'CSV_train.zip' e 'x_test.csv' su Colab!")

In [6]:
# --- CELLA 2: PREPROCESSING ---

# Colonne che contengono le liste (stringhe) da convertire
TS_COLS = ['hr_time_series', 'resp_time_series', 'stress_time_series', 'activity_time_series']

def clean_series_interpolation(series_str):
    """
    Ispirato a 'Accelerometer_Preprocessing.ipynb'.
    Converte stringa in array, rimuove zeri/errori e interpola i dati mancanti.
    """
    if not isinstance(series_str, str):
        return np.array([])

    try:
        # 1. Parsing sicuro
        data = np.array(ast.literal_eval(series_str), dtype=float)

        # 2. Pulizia: valori <= 0 spesso sono errori nel Garmin
        data[data <= 0] = np.nan

        # 3. Interpolazione per mantenere la continuità temporale
        s_data = pd.Series(data)
        s_data = s_data.interpolate(method='linear', limit_direction='both')
        s_data = s_data.fillna(0) # Se tutto è NaN, metti 0

        return s_data.values
    except:
        return np.array([0.0])

def calculate_params(matrix):
    """
    Replica ESATTA di 'datasets.py'.
    Estrae feature statistiche e spettrali da un array 1D.
    """
    if len(matrix) == 0:
        return [0.0] * 6

    # Statistiche nel dominio del tempo
    mean = np.mean(matrix)
    std = np.std(matrix)
    perc25 = np.percentile(matrix, 25)
    perc50 = np.percentile(matrix, 50) # Mediana
    perc75 = np.percentile(matrix, 75)

    # Statistiche nel dominio della frequenza (datasets.py usa FFT)
    f = fft(matrix)
    spEnergy = np.sum(np.abs(f)**2)/len(f)

    return [mean, std, perc25, perc50, perc75, spEnergy]

def process_dataframe(df):
    """
    Trasforma il dataframe con stringhe in un dataset tabellare di feature numeriche.
    """
    extracted_data = []

    # Colonne scalari già presenti (non liste)
    scalar_cols = [c for c in df.columns if 'time_series' not in c and c != 'label' and c != 'id' and 'date' not in c]

    for _, row in df.iterrows():
        row_features = []

        # 1. Aggiungi feature scalari già pronte (es. 'average_stress')
        for col in scalar_cols:
             # Gestione valori mancanti nelle colonne scalari
            val = row[col] if pd.notnull(row[col]) else 0.0
            row_features.append(val)

        # 2. Elabora le serie temporali
        for col in TS_COLS:
            if col in df.columns:
                clean_seq = clean_series_interpolation(row[col])
                feats = calculate_params(clean_seq) # Usa la logica di datasets.py
                row_features.extend(feats)

        extracted_data.append(row_features)

    # Generazione nomi colonne
    col_names = list(scalar_cols)
    suffixes = ['mean', 'std', 'p25', 'p50', 'p75', 'energy']
    for ts_col in TS_COLS:
        if ts_col in df.columns:
            base_name = ts_col.replace('_time_series', '')
            for s in suffixes:
                col_names.append(f"{base_name}_{s}")

    return pd.DataFrame(extracted_data, columns=col_names)

print("Funzioni di preprocessing pronte.")

Funzioni di preprocessing pronte.


In [7]:
# --- CELLA 3: FEDERATED LEARNING CON RANDOM FOREST ---

def run_federated_random_forest():
    """
    Simulazione FL:
    1. Ogni client addestra un Random Forest locale sui suoi dati.
    2. Il server raccoglie tutti gli stimatori (alberi) dai client.
    3. Il Modello Globale è l'unione di tutti gli alberi (Ensemble of Ensembles).
    """
    TRAIN_ROOT = "CSV_train"
    global_estimators = [] # Qui accumuleremo tutti gli alberi di tutti i client

    # Trova le cartelle dei client
    client_folders = []
    for root, dirs, files in os.walk(TRAIN_ROOT):
        csv_files = [f for f in files if f.endswith('.csv')]
        if csv_files:
            client_folders.append(root)

    print(f"Inizio training su {len(client_folders)} gruppi di client...")

    metrics = []

    for i, folder in enumerate(client_folders):
        # A. Caricamento e Fusione Dati del Gruppo (Client)
        files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.csv')]
        df_list = [pd.read_csv(f, sep=';') for f in files]
        if not df_list: continue

        df_client = pd.concat(df_list, ignore_index=True)

        # B. Preprocessing
        if 'label' not in df_client.columns: continue
        y = df_client['label'].values
        X = process_dataframe(df_client)

        # Split locale per validazione interna (opzionale, solo per vedere come va)
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

        # C. Training Locale
        # Usiamo RandomForestRegressor perché il target è continuo (0-100)
        # n_estimators basso per client, poi sommati diventeranno tanti
        clf = RandomForestRegressor(n_estimators=10, max_depth=10, n_jobs=-1, random_state=42)
        clf.fit(X_train, y_train)

        # Valutazione locale
        score = clf.score(X_val, y_val) # R^2 score
        mae = mean_absolute_error(y_val, clf.predict(X_val))
        metrics.append(mae)
        print(f"  Client {i+1}/{len(client_folders)} - MAE Locale: {mae:.2f}")

        # D. "Invio" al server (Accumulo degli alberi)
        # In FL reale, invieremmo i parametri. Qui accumuliamo gli alberi.
        global_estimators.extend(clf.estimators_)

    print(f"\nTraining completato. MAE medio locale: {np.mean(metrics):.2f}")

    # Creiamo un modello "Contenitore" vuoto per usarlo in predizione
    # Deve avere le stesse feature del training
    dummy_model = RandomForestRegressor(n_estimators=len(global_estimators), random_state=42)
    dummy_model.estimators_ = global_estimators
    dummy_model.fit(X_train, y_train) # Fit fittizio solo per inizializzare le classi interne

    return dummy_model

# ESECUZIONE TRAINING
print("Avvio Federated Training...")
global_model = run_federated_random_forest()

Avvio Federated Training...
Inizio training su 9 gruppi di client...
  Client 1/9 - MAE Locale: 9.32
  Client 2/9 - MAE Locale: 9.15
  Client 3/9 - MAE Locale: 11.42
  Client 4/9 - MAE Locale: 12.67
  Client 5/9 - MAE Locale: 10.97
  Client 6/9 - MAE Locale: 9.63
  Client 7/9 - MAE Locale: 11.13
  Client 8/9 - MAE Locale: 9.59
  Client 9/9 - MAE Locale: 13.08

Training completato. MAE medio locale: 10.77


In [8]:
# --- CELLA 4: PREDIZIONE E SUBMISSION ---
if os.path.exists("x_test.csv"):
    print("\nGenerazione predizioni su x_test.csv...")
    df_test = pd.read_csv("x_test.csv", sep=';')

    # 1. Estrazione Feature (Stessa del training)
    X_test = process_dataframe(df_test)

    # Align X_test columns with the features the model was trained on
    # This addresses the ValueError about mismatched feature names
    trained_features = global_model.feature_names_in_

    # Identify missing columns in X_test and add them, filling with 0
    missing_cols = set(trained_features) - set(X_test.columns)
    for c in missing_cols:
        X_test[c] = 0.0

    # Identify extra columns in X_test and drop them
    extra_cols = set(X_test.columns) - set(trained_features)
    if extra_cols:
        X_test = X_test.drop(columns=list(extra_cols))

    # Ensure the order of columns matches the training data
    X_test = X_test[trained_features]

    # 2. Predizione usando il Super-Ensemble
    # Random Forest fa la media delle predizioni di tutti gli alberi accumulati
    preds = global_model.predict(X_test)

    # 3. Salvataggio
    submission = pd.DataFrame({
        'id': df_test['id'],
        'label': preds
    })

    submission.to_csv('submission.csv', index=False)
    print("✅ File 'submission.csv' creato con successo!")
    print(submission.head())
else:
    print("❌ File x_test.csv non trovato.")


Generazione predizioni su x_test.csv...
✅ File 'submission.csv' creato con successo!
   id      label
0   0  58.477778
1   1  61.811111
2   2  60.644444
3   3  61.788889
4   4  62.122222
